# CartPole-v0

http://kvfrans.com/simple-algoritms-for-solving-cartpole/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import gym
env = gym.make('CartPole-v0')
# from gym import wrappers
# env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

[2017-02-19 23:50:18,716] Making new env: CartPole-v0


In [6]:
env.close()
gym.upload('/tmp/cartpole-experiment-1', api_key='sk_uAvJmH4QTBiYO4r9pmQww')
# gym.upload('/tmp/cartpole-experiment-1', writeup='https://gist.github.com/akarazeev/65c72b3a6006a26d02727f1a046bc250', api_key = 'sk_uAvJmH4QTBiYO4r9pmQww')

[2017-02-19 15:49:56,500] [CartPole-v0] Uploading 1 episodes of training data
[2017-02-19 15:49:59,262] [CartPole-v0] Uploading videos of 1 training episodes (9370 bytes)
[2017-02-19 15:50:00,817] [CartPole-v0] Creating evaluation object from /tmp/cartpole-experiment-1 with learning curve and training video
[2017-02-19 15:50:01,815] 
****************************************************
You successfully uploaded your evaluation on CartPole-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_5zP1dcS4uVOKQVhV9bzA

****************************************************


In [2]:
actions = []
for i_episode in range(10):
    observation = env.reset()
#     parameters = np.random.rand(4) * 2 - 1
    parameters = [ 0.72499273 , -0.17340878 , -0.18721514 , 0.48691021]
    for t in range(100):
        env.render()
        action = 0 if np.matmul(parameters, observation) < 0 else 1
        observation, reward, done, info = env.step(action)
        actions.append(action)
        if done:
            print("Episode finished after {} timesteps with\
                    parameters {}".format(t+1, parameters))
            break
env.close()

[2017-02-19 15:32:05,806] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.32839.video000000.mp4
[2017-02-19 15:32:10,969] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.32839.video000001.mp4


Episode finished after 97 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 46 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 49 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 68 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 87 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 91 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 71 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]


[2017-02-19 15:32:20,041] Starting new video recorder writing to /tmp/cartpole-experiment-1/openaigym.video.0.32839.video000008.mp4


Episode finished after 81 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]
Episode finished after 49 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]


[2017-02-19 15:32:22,762] Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/cartpole-experiment-1')


Episode finished after 59 timesteps with                    parameters [0.72499273, -0.17340878, -0.18721514, 0.48691021]


In [2]:
nsteps = 200

def run_episode(env, parameters):
    observation = env.reset()
    totalreward = 0
    for _ in range(nsteps):
        env.render()
        action = 0 if np.matmul(parameters, observation) < 0 else 1
        observation, reward, done, info = env.step(action)
        totalreward += reward
        if done:
            break
#     env.close()
    return totalreward

## 1. The random guessing algorithm

In [3]:
bestparams = None
bestreward = 0

env = gym.make('CartPole-v0')
for _ in range(10000):
    parameters = np.random.rand(4) * 2 - 1
    tmpreward = run_episode(env, parameters)
    print(tmpreward)
    if tmpreward > bestreward:
        bestreward = tmpreward
        bestparams = parameters
        
        if tmpreward == nsteps:
            break
env.close()

[2017-02-19 23:50:25,484] Making new env: CartPole-v0


48.0
8.0
21.0
29.0
52.0
20.0
122.0
9.0
82.0
10.0
9.0
9.0
200.0


In [4]:
bestparams

array([ 0.03821743,  0.83757862,  0.88963255,  0.62922493])

## 2. The hill-climbing algorithm

In [5]:
noise_scale = 1
bestparams = np.random.rand(4) * 2 - 1
bestreward = run_episode(env, bestparams)

env = gym.make('CartPole-v0')
for _ in range(100):
    noise = (np.random.rand(4) * 2 - 1) * noise_scale
    parameters = bestparams + noise
    tmpreward = run_episode(env, parameters)
    print(tmpreward, bestreward)
    if tmpreward > bestreward:
        bestreward = tmpreward
        bestparams = parameters
        
        if tmpreward == nsteps:
            break
env.close()

[2017-02-19 23:50:51,674] Making new env: CartPole-v0


87.0 53.0
10.0 87.0
200.0 87.0


## 3. Policy gradient algorithm

In [1]:
import tensorflow as tf
import numpy as np
import gym
env = gym.make('CartPole-v0')

[2017-02-19 18:05:51,814] Making new env: CartPole-v0


In [2]:
def policy_gradient():
    params = tf.get_variable("policy_parameters", [4,2])
    state = tf.placeholder("float", [None,4])
    actions = tf.placeholder("float", [None,2])
    linear = tf.matmul(state, params)
    probabilities = tf.nn.softmax(linear)
    good_probabilities = tf.reduce_sum(tf.mul(probabilities, actions), reduction_indices=[1])
    
    log_probabilites = tf.log(good_probabilities)
    loss = -tf.reduce_sum(log_probabilites)
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
    
    return probabilities, state
    
def value_gradient():
    state = tf.placeholder("float", [None,4])
    w1 = tf.get_variable("w1", [4,10])
    b1 = tf.get_variable("b1", [10])
    h1 = tf.nn.relu(tf.matmul(state, w1) + b1)
    w2 = tf.get_variable("w2", [10,1])
    b2 = tf.get_variable("b2", [1])
    calculated = tf.matmul(h1, w2) + b2
    
    newvals = tf.placeholder("float", [None,1])
    diffs = calculated - newvals
    loss = tf.nn.l2_loss(diffs)
    optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

In [3]:
pl_probabilities, pl_state = policy_gradient()
observation = env.reset()
actions = []
transitions = []
totalreward = 0
for _ in range(200):
    obs_vector = np.expand_dims(observation, axis=0)
    with tf.Session() as sess:
        probs = sess.run(pl_probabilities, feed_dict={pl_state: obs_vector})
    action = 0 if random.uniform(0,1) < probs[0][0] else 1
    
    states.append(observation)
    actionblank = np.zeros(2)
    actionblank[action] = 1
    actions.append(actionblank)
    
    old_observation = observation
    observation, reward, done, info = env.step(action)
    transitions.append((old_observation, action, reward))
    totalreward += reward
    
    if done:
        break

FailedPreconditionError: Attempting to use uninitialized value policy_parameters
	 [[Node: policy_parameters/read = Identity[T=DT_FLOAT, _class=["loc:@policy_parameters"], _device="/job:localhost/replica:0/task:0/cpu:0"](policy_parameters)]]

Caused by op 'policy_parameters/read', defined at:
  File "/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.5.2_3/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File